In [15]:
"""""
simulation of a workshop
in this workshop we have 2 kind of orders, order 1 and order 2
enterarrival time of order 1 distribution ~ exponential with mean of 7/6 hours
enterarrivaltime of order 2 distribution ~ exponential with mean of 1/4 hours
service time of A machine distribution ~ uniform between 35/60 to 65/60 hours
service time of B machine distribution ~ uniform between 3/60 to 13/60 hours
service time of C machine distribution ~ Triangular of 6/60,8/60,10/60 hours
working time of A machine distribution ~ 100x/60+50/60 that x has weibul distribution with shape of 2 and scale of 1
working time of B machine distribution ~  uniform between 50/60 to 350/60 hours
repairing time of A distribution ~ uniform between 1/60 to 29/60 hours
repairing time of B distribution ~ uniform between 2/60 to 18/60 hours
No limit on Queue's length
People get service in a FIFO system
outputs: 1: compeleted orders
        2: servers(A, B & C)efficiency
        3: Queue waiting time mean for A,B & C 
        4: Time weighted length mean for A, B & C
        5: beirng in the system time mean
starting state: system is empty           
authurs:Mehrdad Moradi, Behzad Yaghubi
date: 2020-12-3
"""""
#modules needed 
import numpy as num
import random
import math
from math import sqrt
import xlsxwriter as xw      # Used to create Excel Directly
from numpy import mean
import statsmodels.stats.api as sms



#functions




#starting_state function 
def starting_state():
    
    state=dict() #state: defining variable to put state variables
    
    state['server condition']={'A': 1,'B': 1,'C': 0} #state variables for servers
    
    state['queues']={'A': [],
                     'B': [],
                     'Cord1': [],
                     'Cord2': []}       # we save the queues as lists of orders
    
    state['helping']={'q1':-1,
                       'q2':-1,
                        'stockorder1':'null', 
                         'stockorder2':'null'} #we use these for when A or B get out of service
    
    
 

    #data collecting: used to collect datas we need to collect, used in calculating cumulative statistics
    data_collecting=dict() 
    
    data_collecting['clock']=0 #we use this clock to calculate time weighted length for queues
    
    data_collecting['order1s']=dict()
    data_collecting['order2s']=dict() #we save order1s or order2s as dict, keys are the name of order like order11 and the 
                                       #value is the list of times, #    this list saves these times in this
                                        #order as a list: entering to system, begining of service A,begining of service B 
                                          #begining of service C,end of service A, end of service B, end of service C,
                                           #last stop of service time,last back to service time
    
    # Cumulative statistics   
      #cumulative statistics needed
    cumulative_stat = {'compeleted orders':0 ,
                       'served by':{'A':0,'B':0,'Cord1':0,'Cord2':0},
                       'server busy time':{'A':0,'B':0,'C':0 },
                       'waiting in queue':{'A':0,'B':0,'Cord1':0,'Cord2':0},
                       'time weighted length':{'A':0,'B':0,'Cord1':0,'Cord2':0},
                       'total being in system':{'order1':0,'order2':0},
                       'stock_num':{'A':0,'B':0}
                      }


    #future-event-list
    future_event_list = list()
    fel_maker(future_event_list, 'arrival1', 0, 'order11')        # Difference: make an entrance of specific order1 (order11)
    fel_maker(future_event_list, 'arrival2', 0, 'order21')        # Difference: make an entrance of specific order2 (order21)
    fel_maker(future_event_list, 'out of service A',0,'A')      #at t=0: A start working and we should generate an out of service event fo A
    fel_maker(future_event_list, 'out of service B',0,'B')      #at t=0: B start working and we should generate an out of service event fo B

    return state, data_collecting, future_event_list, cumulative_stat



#fel_maker: when we need to make a new event, we use this function to make new event(based on activity dist.) and update 
            #future event list

def fel_maker(future_event_list,event_type,clock,order):
    
    #next 4 global variables will be used for sensitivity analysis, at first we should let them be 1, for sensitivity 
      #analysis we change them to change the parameters
    global arrB
    global work_A
    global rep_A
    global serv_C
    
    rand_num = random.random() #making random number, we use inverse transform method to generate random variates
    event_time=0 #defining event_time, we change it later

    
    if event_type=='arrival1':
        event_time=clock+round((-7/6)*math.log(rand_num),3)


    if event_type=='arrival2':
        event_time=clock+round(arr_B*((-1/4)*math.log(rand_num)),3)


    if event_type=='end of service A':
        event_time=clock+round((35/60)+(30/60)*rand_num,3)


    if event_type=='end of service B':
        event_time=clock+round((3/60)+(10/60)*rand_num,3)


    if event_type=='end of service C':

        if rand_num<=(0.5):
            event_time=clock+round(serv_C*((sqrt(2*rand_num)+3)/30),3)
        else:
            event_time=clock+round(serv_C*((5-sqrt(2*(1-rand_num)))/30),3)



    if event_type=='out of service A':
            event_time=clock+round(work_A*(50/60+(100/60)*math.sqrt(-math.log(rand_num))),3)


    if event_type=='out of service B':
        event_time=clock+round(50/60+(300/60)*rand_num,3)


    if event_type=='being repaired A':
        event_time=clock+round(rep_A*(1/60+(28/60)*rand_num),3)


    if event_type=='being repaired B':
        event_time=clock+round(2/60+(16/60)*rand_num,3)



    new_event = {'event type': event_type,
                 'event time': event_time,
                 'order or server': order}    # additional element in event notices (order No.)

    future_event_list.append(new_event)





        
        
        
#event functions: when we call these functions,its like that special event has happend  
        #and every thing that need to change(state,data_collecting and cumulative_stat),
                #will change.its like we translate our dynamic diagram into python codes


def arrival1(future_event_list, state, data_collecting, clock, order1, cumulative_stat):
    #first of all, we update time weighted length
    
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    data_collecting['order1s'][order1]=[0]*9 #dafining the list to save times
    data_collecting['order1s'][order1][0]=clock#saves the entering to system time for this order
    
    
     #figure next number of order1
    order1_num = int(order1[6:])
    order1_num+= 1
    fel_maker(future_event_list,'arrival1',clock,'order1'+str(order1_num)) #make new arrival for order 1
    
    
    #based on dynamic diagram(dynamic description)
    
    if state['server condition']['A']==1:
        state['server condition']['A']+=1
        fel_maker(future_event_list,'end of service A',clock,order1)
        data_collecting['order1s'][order1][1]=clock #saving the begining of service A time for this order
       


    else:
        state['queues']['A'].append(order1)

        #other changes in data_collecting and cumulative_stat




    
    data_collecting['clock']=clock#updating time        




                  
def arrival2(future_event_list, state, data_collecting, clock, order2, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    data_collecting['order2s'][order2]=[0]*9 #dafining the list to save times
    data_collecting['order2s'][order2][0]=clock#saves the entering to system time for this order

    
     #figure out how to find next number of order2
    order2_num = int(order2[6:])
    order2_num+= 1
    fel_maker(future_event_list,'arrival2',clock,'order2'+str(order2_num))  #make new arrival for order 2
    
     #based on dynamic diagram(dynamic description)
    if state['server condition']['B']==1:
        state['server condition']['B']+=1
        fel_maker(future_event_list,'end of service B',clock,order2)
        data_collecting['order2s'][order2][2]=clock#saving the begining of service A time for this order


    else:
        state['queues']['B'].append(order2)

            #other changes in data_collecting and cumulative_stat
   
    data_collecting['clock']=clock#updating time        

                  
        
            
def end_of_service_A(future_event_list, state, data_collecting, clock, order1, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])

        
        
        
    #based on diagram:
              #backward look
    if len(state['queues']['A'])==0:
        #nessesary changes to state, data_collecting,cumulative_stat:
        state['server condition']['A']=1
        data_collecting['order1s'][order1][4]=clock
        cumulative_stat['server busy time']['A']+=data_collecting['order1s'][order1][4]-data_collecting['order1s'][order1][1]
        cumulative_stat['served by']['A']+=1
   


    else:
        #nessesary changes to state, data_collecting,cumulative_stat:
        #new one enters from the queue to server A
        order=state['queues']['A'][0]
        state['queues']['A'].pop(0)
        fel_maker(future_event_list,'end of service A',clock,order)
        data_collecting['order1s'][order][1]=clock
        
        cumulative_stat['waiting in queue']['A']+=clock-data_collecting['order1s'][order][0]
        data_collecting['order1s'][order1][4]=clock 
        cumulative_stat['server busy time']['A']+=data_collecting['order1s'][order1][4]-data_collecting['order1s'][order1][1]
        cumulative_stat['served by']['A']+=1
      
        


    #forward look  
    if state['server condition']['C']==0:
        #nessesary changes to state, data_collecting,cumulative_stat:
        #state
        state['server condition']['C']=1
        fel_maker(future_event_list,'end of service C',clock,order1)
        #data_collecting&cumulative_stat
        data_collecting['order1s'][order1][3]=clock
        
        #nothing happen to cumulative_stat



    else:
        #state
        state['queues']['Cord1'].append(order1)
       
        
        
    data_collecting['clock']=clock#updating clock of data_collecting to current clock



def end_of_service_B(future_event_list, state, data_collecting, clock, order2, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    
    
    
    #based on diagram:
          #backward look
    if len(state['queues']['B'])==0:
          #nessesary changes to state, data_collecting,cumulative_stat:
        state['server condition']['B']=1
        data_collecting['order2s'][order2][5]=clock
        cumulative_stat['server busy time']['B']+=data_collecting['order2s'][order2][5]- data_collecting['order2s'][order2][2]
        cumulative_stat['served by']['B']+=1
        



    else:
        #nessesary changes to state, data_collecting,cumulative_stat:
        #new one enters from the queue to server A
        order=state['queues']['B'][0]
        state['queues']['B'].pop(0)
        fel_maker(future_event_list,'end of service B',clock,order)
        data_collecting['order2s'][order][2]=clock
        
        cumulative_stat['waiting in queue']['B']+=clock-data_collecting['order2s'][order][0]
        data_collecting['order2s'][order2][5]=clock 
        cumulative_stat['server busy time']['B']+=data_collecting['order2s'][order2][5]- data_collecting['order2s'][order2][2]
        cumulative_stat['served by']['B']+=1
 
       



    #forward look  
    if state['server condition']['C']==0:
        #nessesary changes to state, data_collecting,cumulative_stat:
        #state
        state['server condition']['C']=1
        fel_maker(future_event_list,'end of service C',clock,order2)
        #data_collecting&cumulative_stat
        data_collecting['order2s'][order2][3]=clock
      
        #nothing happen to cumulative_stat



    else:
        #state
        state['queues']['Cord2'].append(order2)
       
        
        
    data_collecting['clock']=clock#updating clock of data_collecting to current clock

              
def end_of_service_C(future_event_list, state, data_collecting, clock, order, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
       
    if len(state['queues']['Cord1'])==0:
        if len(state['queues']['Cord2'])==0:
            state['server condition']['C']=0

        else:

            orderr=state['queues']['Cord2'][0]#note that this is another order entering to C
         
            state['queues']['Cord2'].pop(0)
           
            fel_maker(future_event_list,'end of service C',clock,orderr)
            #data_collecting & cumulative_stat 
            data_collecting['order2s'][orderr][3]=clock
            cumulative_stat['waiting in queue']['Cord2']+=data_collecting['order2s'][orderr][3]- data_collecting['order2s'][orderr][5]
           
    else:     

        orderr=state['queues']['Cord1'][0]#note that this is another order entering to C
        state['queues']['Cord1'].pop(0)
       
        fel_maker(future_event_list,'end of service C',clock,orderr)
        
        #data_collecting & cumulative_stat 
        data_collecting['order1s'][orderr][3]=clock 
        cumulative_stat['waiting in queue']['Cord1']+=data_collecting['order1s'][orderr][3]- data_collecting['order1s'][orderr][4]
    

    #data_collecting & cumulative_stat         
    if order[5]=='1':
        data_collecting['order1s'][order][6]=clock
        cumulative_stat['total being in system']['order1']+=data_collecting['order1s'][order][6]-data_collecting['order1s'][order][0]
        cumulative_stat['server busy time']['C']+=data_collecting['order1s'][order][6]- data_collecting['order1s'][order][3]
        cumulative_stat['served by']['Cord1']+=1
    else:
        data_collecting['order2s'][order][6]=clock
        cumulative_stat['total being in system']['order2']+=data_collecting['order2s'][order][6]-data_collecting['order2s'][order][0]
        cumulative_stat['server busy time']['C']+=data_collecting['order2s'][order][6]- data_collecting['order2s'][order][3]
        cumulative_stat['served by']['Cord2']+=1
        
    data_collecting['clock']=clock#updating clock of data_collecting to current clock         

    cumulative_stat['compeleted orders']+=1
    


                  
def out_of_service_A(future_event_list, state, data_collecting, clock, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    
   
    
    
    
    #based on diagram
    if state['server condition']['A']==2:
        #now we want to find end of service A element for the order A was working on
        index=next((i for i, item in enumerate(future_event_list) if item['event type'] =='end of service A'), None)
        #witch order was that A was working on?
        state['helping']['stockorder1']=future_event_list[index]['order or server']
        state['helping']['q1']=future_event_list[index]['event time']-clock
        #removing that event from FEL
        future_event_list.pop(index)
        #adding new order in queue
        state['queues']['A'].insert(0,state['helping']['stockorder1'])
        #data_collecting
        data_collecting['order1s'][state['helping']['stockorder1']][7]=clock
        #maybe need some cumulative_stat, check it later!
        cumulative_stat['stock_num']['A']+=1
        


    state['server condition']['A']=0   
    fel_maker(future_event_list,'being repaired A',clock,'A')
              #other changes in data_collecting and cumulative_stat

    data_collecting['clock']=clock#updating time        
   

                  
                
              
def out_of_service_B(future_event_list, state, data_collecting, clock, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    
    
    #based on diagram
    if state['server condition']['B']==2:
        #now we want to find end of service A element for the order B was working on
        index=next((i for i, item in enumerate(future_event_list) if item['event type'] =='end of service B'), None)
        #witch order was that A was working on?
        state['helping']['stockorder2']=future_event_list[index]['order or server']
        state['helping']['q2']=future_event_list[index]['event time']-clock
        #removing that event from FEL
        future_event_list.pop(index)
        #adding new order in queue
        state['queues']['B'].insert(0,state['helping']['stockorder2'])
        #data_collecting
        data_collecting['order2s'][state['helping']['stockorder2']][7]=clock
        #maybe need some sumulative_stat, check it later!
        cumulative_stat['stock_num']['B']+=1

    state['server condition']['B']=0   
    fel_maker(future_event_list,'being repaired B',clock,'B')
              #other changes in data_collecting and cumulative_stat
  
    data_collecting['clock']=clock#updating time        

         
                  
              
def being_repaired_A(future_event_list, state, data_collecting, clock, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    
        #making next out of service event
    fel_maker(future_event_list,'out of service A',clock,'A')
   
    if state['helping']['q1']==-1:
        if len(state['queues']['A'])==0:
            state['server condition']['A']=1

        else:
            state['server condition']['A']=2
            order=state['queues']['A'][0]
            state['queues']['A'].pop(0)
            fel_maker(future_event_list,'end of service A',clock,order)
            #data_collecting & cumulative_stat
            data_collecting['order1s'][order][1]=clock
            cumulative_stat['waiting in queue']['A']+=data_collecting['order1s'][order][1]- data_collecting['order1s'][order][0]
         




    else:



        future_event_list.append({'event type': 'end of service A','event time': clock+state['helping']['q1'],'order or server': state['helping']['stockorder1']})
        state['server condition']['A']=2
        #remove that stock order from A queue
        state['queues']['A'].pop(0)
        #data_collecting & cumulative_stat
      
        data_collecting['order1s'][state['helping']['stockorder1']][8]=clock
        cumulative_stat['server busy time']['A']-=(data_collecting['order1s'][state['helping']['stockorder1']][8]-data_collecting['order1s'][state['helping']['stockorder1']][7])
        cumulative_stat['waiting in queue']['A']+=data_collecting['order1s'][state['helping']['stockorder1']][8]-data_collecting['order1s'][state['helping']['stockorder1']][7]
        

    data_collecting['clock']=clock#updating clock of data_collecting to current clock
    #we don't have any atock order 1 anymore!
    state['helping']['stockorder1']='null'
    state['helping']['q1']=-1


              
def being_repaired_B(future_event_list, state, data_collecting, clock, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
        #making next out of service event
    fel_maker(future_event_list,'out of service B',clock,'B')

    if state['helping']['q2']==-1:
        if len(state['queues']['B'])==0:
            state['server condition']['B']=1

        else:
            state['server condition']['B']=2
            order=state['queues']['B'][0]
            state['queues']['B'].pop(0)
            fel_maker(future_event_list,'end of service B',clock,order)
            #data_collecting & cumulative_stat
            data_collecting['order2s'][order][2]=clock
            cumulative_stat['waiting in queue']['B']+=data_collecting['order2s'][order][2]-data_collecting['order2s'][order][0]
          





    else:



        future_event_list.append({'event type': 'end of service B',
        'event time': clock+state['helping']['q2'],'order or server': state['helping']['stockorder2']})
        state['server condition']['B']=2
        #remove that stock order from A queue
        state['queues']['B'].pop(0)
        #data_collecting & cumulative_stat
        data_collecting['order2s'][state['helping']['stockorder2']][8]=clock
        cumulative_stat['server busy time']['B']-=data_collecting['order2s'][state['helping']['stockorder2']][8]-data_collecting['order2s'][state['helping']['stockorder2']][7]
        cumulative_stat['waiting in queue']['B']+=data_collecting['order2s'][state['helping']['stockorder2']][8]-data_collecting['order2s'][state['helping']['stockorder2']][7]



    data_collecting['clock']=clock #updating clock of data_collecting to current clock
    #we don't have any atock order 1 anymore!
    state['helping']['stockorder2']='null'
    state['helping']['q2']=-1



    
#next tow functions will be used to write in trace excel   


def end_of_simulation(state, data_collecting, clock, cumulative_stat):
    #first of all, we update time weighted length
    cumulative_stat['time weighted length']['A']+=(len(state['queues']['A']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['B']+=(len(state['queues']['B']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord1']+=(len(state['queues']['Cord1']))*(clock- data_collecting['clock'])
    cumulative_stat['time weighted length']['Cord2']+=(len(state['queues']['Cord2']))*(clock- data_collecting['clock'])
    
    #server busy time in cumulative stat
    if(state['server condition']['A']==2):#A
        cumulative_stat['server busy time']['A']+=clock-data_collecting['clock']
    if(state['server condition']['B']==2):#B
        cumulative_stat['server busy time']['B']+=clock-data_collecting['clock']
    if(state['server condition']['C']==1):#C
        cumulative_stat['server busy time']['C']+=clock-data_collecting['clock']
        
    #waiting in queue in cumulative stat
    #A
    if(len(state['queues']['A'])!=0):
        for i in range(len(state['queues']['A'])):
            cumulative_stat['waiting in queue']['A']+=data_collecting['clock']-data_collecting['order1s'][state['queues']['A'][i]][0]
    #B
    if(len(state['queues']['B'])!=0):
        for j in range(len(state['queues']['B'])):
            cumulative_stat['waiting in queue']['B']+=data_collecting['clock']-data_collecting['order2s'][state['queues']['B'][j]][0]
   
    #Cord1 
    #in addition, we want to calculate total being in system too
                       
    if(len(state['queues']['Cord1'])!=0):
        for i in range(len(state['queues']['Cord1'])):
            cumulative_stat['waiting in queue']['Cord1']+=data_collecting['clock']-data_collecting['order1s'][state['queues']['Cord1'][i]][4]              
            cumulative_stat['total being in system']['order1']+=data_collecting['clock']-data_collecting['order1s'][state['queues']['Cord1'][i]][0]         
                       
    #Cord2 
    #in addition, we want to calculate total being in system too
                       
    if(len(state['queues']['Cord2'])!=0):
        for i in range(len(state['queues']['Cord2'])):
            cumulative_stat['waiting in queue']['Cord2']+=data_collecting['clock']-data_collecting['order2s'][state['queues']['Cord2'][i]][5]              
            cumulative_stat['total being in system']['order2']+=data_collecting['clock']-data_collecting['order2s'][state['queues']['Cord2'][i]][0]
                       
                       
     #update clock
    data_collecting['clock']=clock
                       
def output_excel(worksheet, future_event_list, state,cumulative_stat,row_num):

    global max_fel
    global header_list      # we update the header list in this function
    future_event_list1 = sorted(future_event_list, key=lambda x: x['event time'])
    
    new_row = [row_num, future_event_list1[0]['event type'], future_event_list1[0]['event time'],
               state['server condition']['A'],state['server condition']['B'],state['server condition']['C'],
               len(state['queues']['A']), len(state['queues']['B']), len(state['queues']['Cord1']),
               len(state['queues']['Cord2']),state['helping']['stockorder1'],
               state['helping']['stockorder2'],state['helping']['q1'],state['helping']['q2'],
               cumulative_stat['compeleted orders'],cumulative_stat['served by']['A'],
               cumulative_stat['served by']['B'],cumulative_stat['served by']['Cord1'],
               cumulative_stat['served by']['Cord2'],cumulative_stat['server busy time']['A'],
               cumulative_stat['server busy time']['B'],cumulative_stat['server busy time']['C'],
               cumulative_stat['waiting in queue']['A'],cumulative_stat['waiting in queue']['B'],
               cumulative_stat['waiting in queue']['Cord1'],cumulative_stat['waiting in queue']['Cord2'],
               cumulative_stat['time weighted length']['A'],cumulative_stat['time weighted length']['B'],
               cumulative_stat['time weighted length']['Cord1'],cumulative_stat['time weighted length']['Cord2'],
               cumulative_stat['total being in system']['order1'],cumulative_stat['total being in system']['order2'],
               cumulative_stat['stock_num']['A'],cumulative_stat['stock_num']['B']]             # Creating new row

  
    
    
    
    # Update the header list and max_fel
    if len(future_event_list1)-1 > max_fel:
        for fel_counter in range(max_fel, len(future_event_list1) - 1):
            header_list.extend(("Future Event Type "+str(fel_counter+1), "Future Event Time "+str(fel_counter+1)))
        max_fel = len(future_event_list1) - 1

    else:
        for add_number in range(max_fel - (len(future_event_list1) - 1)):
            future_event_list1.append({"event type": "", "event time": ""})

    for evno in future_event_list1[1:]:
        new_row.extend((evno['event type'], evno['event time']))

    for col in range(len(header_list)):
        worksheet.write(0, col, header_list[col])
        worksheet.write(row_num, col, new_row[col])
        
        
        
#seting the format of excel as we want        
def excel_formatting(workbook, worksheet, row_num):

    cell_format_header = workbook.add_format()
    cell_format_header.set_align('center')
    cell_format_header.set_align('vcenter')
    cell_format_header.set_font('Times New Roman')
    cell_format_header.set_bg_color('#e2ac8d')  # Find the color in Excel use RGB to HEX online tool
    cell_format_header.set_bold(True)
    
    worksheet.set_row(0, None, cell_format_header)

    worksheet.set_column(0, 0, 20)
    worksheet.set_column(1, 1, 13)
    worksheet.set_column(2, 2, 9)
    worksheet.set_column(3, 5, 8)
    worksheet.set_column(6, 9, 6)
    worksheet.set_column(10, 9 + 2*max_fel, 19)#set the size of cells
    cell_format = workbook.add_format()
    cell_format.set_align('center')
    cell_format.set_font('Times New Roman')
    for row in range(row_num):
        worksheet.set_row(row+1, None, cell_format)
      
        
        
        


#simulation
def simulation(simulation_time):
    #next 4 global variables will be used in sensitivity analysis
    global arrB
    global work_A
    global rep_A
    global serv_C
    
    global sen_analysis#this global list will save the outputs for each parameter value in sensivity analysis
    utility_A=[0]*100
    utility_B=[0]*100
    utility_C=[0]*100
    mean_lq_A=[0]*100
    mean_lq_B=[0]*100
    mean_lq_Cord1=[0]*100
    mean_lq_Cord2=[0]*100                    #we run the simulation for 100 times and save the outputs in these lists
    mean_lq_C=[0]*100
    mean_wq_A=[0]*100
    mean_wq_B=[0]*100
    mean_wq_C=[0]*100
    mean_wq_Cord1=[0]*100
    mean_wq_Cord2=[0]*100
    mean_being_in_system_order1=[0]*100
    mean_being_in_system_order2=[0]*100
    mean_being_in_system_all_orders=[0]*100
    compeleted_orders=[0]*100
    
    
    for i in range(100):
        
        #note when we want to do sensitivity analysis we comment ** codes, because we dont want to rewrite trace excel
        workbook = xw.Workbook("C:/Users/System-Pc/Desktop/handy/sut/ترم 7/simulation/پروژه/فاز 2/Output_XW.xlsx")  #**
        # Open a New Workbook (Simultaneously Created an Excel) #note that if you want run this in your computer you should
                                                                  #change the address
            
#         worksheet = workbook.add_worksheet("production workshop simulation")   # Add a Sheet  #**
#         row_num = 1            # Define row number  #**
        
        state, data_collecting, future_event_list, cumulative_stat = starting_state()          # Starting state
        clock = 0
        
        # Add specific events
        future_event_list.append({'event type': 'end of simulation', 'event time': simulation_time, 'order or server':'null'})  

        # Continue till the simulation time ends
        while clock <= simulation_time:      

            sorted_fel = sorted(future_event_list, key=lambda x: x['event time'])     # Sort the FEL based on event times
            current_event = sorted_fel[0]               # The first element is what happening now
            clock = current_event['event time']         # Move the time forward


            if clock <= simulation_time:

                current_order=current_event['order or server']
                if current_event['event type'] == 'arrival1':
                      arrival1(future_event_list, state, data_collecting, clock, current_order, cumulative_stat)
                if current_event['event type'] == 'arrival2':
                      arrival2(future_event_list, state, data_collecting, clock, current_order, cumulative_stat)
                if current_event['event type'] == 'end of service A':
                      end_of_service_A(future_event_list, state, data_collecting, clock, current_order, cumulative_stat)
                if current_event['event type'] == 'end of service B':
                      end_of_service_B(future_event_list, state, data_collecting, clock, current_order, cumulative_stat)
                if current_event['event type'] == 'end of service C':
                      end_of_service_C(future_event_list, state, data_collecting, clock, current_order, cumulative_stat)
                if current_event['event type'] == 'out of service A':
                      out_of_service_A(future_event_list, state, data_collecting, clock, cumulative_stat)
                if current_event['event type'] == 'out of service B':
                      out_of_service_B(future_event_list, state, data_collecting, clock, cumulative_stat)
                if current_event['event type'] == 'being repaired A':
                      being_repaired_A(future_event_list, state, data_collecting, clock, cumulative_stat)
                if current_event['event type'] == 'being repaired B':
                      being_repaired_B(future_event_list, state, data_collecting, clock, cumulative_stat)
                if current_event['event type'] == 'end of simulation':
                      end_of_simulation(state, data_collecting, clock, cumulative_stat)
                        
#                 output_excel(worksheet, future_event_list, state,cumulative_stat,row_num)    # Use the xlsxwriter Module  #**
#                 row_num += 1        # Preparing row number for the next time  #**

                  #adjust future_event_list
                future_event_list.remove(current_event)
            else:
                future_event_list.clear()#clearing future event list after simulation ends

#         excel_formatting(workbook, worksheet, row_num)         # Format the Output_XW Excel #**
#         workbook.close()              # Close the workbook  #**




        # Calculating the outputs
        #desired by management
        compeleted_orders[i]=cumulative_stat['compeleted orders']
        
        
        utility_A[i]=cumulative_stat['server busy time']['A']/simulation_time
        utility_B[i]=cumulative_stat['server busy time']['B']/simulation_time
        utility_C[i]=cumulative_stat['server busy time']['C']/simulation_time  
    
        mean_lq_A[i]=cumulative_stat['time weighted length']['A']/simulation_time
        mean_lq_B[i]=cumulative_stat['time weighted length']['B']/simulation_time

        mean_lq_Cord1[i]=cumulative_stat['time weighted length']['Cord1']/simulation_time
        mean_lq_Cord2[i]=cumulative_stat['time weighted length']['Cord2']/simulation_time
        mean_lq_C[i]=mean_lq_Cord1[i]+mean_lq_Cord2[i]

        mean_wq_A[i]=cumulative_stat['waiting in queue']['A']/len(data_collecting['order1s'])
        mean_wq_B[i]=cumulative_stat['waiting in queue']['B']/len(data_collecting['order2s'])
        mean_wq_C[i]=(cumulative_stat['waiting in queue']['Cord1']+cumulative_stat['waiting in queue']['Cord2'])/(cumulative_stat['served by']['A']+
                                                                                                       cumulative_stat['served by']['B'])
        mean_wq_Cord1[i]=cumulative_stat['waiting in queue']['Cord1']/(cumulative_stat['served by']['A'])
        mean_wq_Cord2[i]=cumulative_stat['waiting in queue']['Cord2']/(cumulative_stat['served by']['B'])


        #suggested by our team
        mean_being_in_system_order1[i]=cumulative_stat['total being in system']['order1']/len(data_collecting['order1s'])
        mean_being_in_system_order2[i]=cumulative_stat['total being in system']['order2']/len(data_collecting['order2s'])
        mean_being_in_system_all_orders[i]=((cumulative_stat['total being in system']['order1']+cumulative_stat['total being in system']['order2'])/(len(data_collecting['order1s'])+len(data_collecting['order2s'])))

    
#     print  all outputs

#     print('outputs desired by management:')
#     print('compeleted orders   ='+str(mean(compeleted_orders))+'\nutility A   ='+str(mean(utility_A))
#     +'\nutility B   ='+str(mean(utility_B))+'\nutility C   ='+str(mean(utility_C))
#     +'\nmean lq A   ='+str(mean(mean_lq_A))+'\nmean lq B   ='+str(mean(mean_lq_B))+'\nmean lq C   ='+str(mean(mean_lq_C))
#     +'\nmean lq Cord1   ='+str(mean(mean_lq_Cord1))+'\nmean lq Cord2   ='+str(mean(mean_lq_Cord2))
#     +'\nmean wq A   ='+str(mean(mean_wq_A))+'\nmean wq B   ='+str(mean(mean_wq_B))+'\nmean wq C  ='+str(mean(mean_wq_C))
#     +'\nmean wq Cord1   ='+str(mean(mean_wq_Cord1))+'\nmean wq Cord2   ='+str(mean(mean_wq_Cord2)))
    
    
#     print('suggested by our team:')
#     print('mean being in system for order1   ='+str(mean(mean_being_in_system_order1)))
#     print('mean being in system for order2   ='+str( mean(mean_being_in_system_order2)))
#     print('mean being in system for all orders  ='+str( mean(mean_being_in_system_all_orders)) )                                 
#     print('end!')
#     print('CI:')
#     print('compeleted orders '+str(sms.DescrStatsW(compeleted_orders).tconfint_mean()))
#     print('utility_A'+str(sms.DescrStatsW(utility_A).tconfint_mean()))
#     print('utility_B'+str(sms.DescrStatsW(utility_B).tconfint_mean()))
#     print('utility_C'+str(sms.DescrStatsW(utility_C).tconfint_mean()))
#     print('mean lq A'+str(sms.DescrStatsW(mean_lq_A).tconfint_mean()))
#     print('mean lq B'+str(sms.DescrStatsW(mean_lq_B).tconfint_mean()))
#     print('mean lq C'+str(sms.DescrStatsW(mean_lq_C).tconfint_mean()))
#     print('mean lq Cord1'+str(sms.DescrStatsW(mean_lq_Cord1).tconfint_mean()))
#     print('mean lq Cord2'+str(sms.DescrStatsW(mean_lq_Cord2).tconfint_mean()))
#     print('mean wq A'+str(sms.DescrStatsW(mean_wq_A).tconfint_mean()))
#     print('mean wq B'+str(sms.DescrStatsW(mean_wq_B).tconfint_mean()))
#     print('mean wq C'+str(sms.DescrStatsW(mean_wq_C).tconfint_mean()))
#     print('mean wq Cord1'+str(sms.DescrStatsW(mean_wq_Cord1).tconfint_mean()))
#     print('mean wq Cord2'+str(sms.DescrStatsW(mean_wq_Cord2).tconfint_mean()))
#     print('mean being in system for order1 '+str(sms.DescrStatsW(mean_being_in_system_order1).tconfint_mean()))
#     print('mean being in system for order2 '+str(sms.DescrStatsW(mean_being_in_system_order2).tconfint_mean()))
#     print('mean being in system for all orders'+str(sms.DescrStatsW(mean_being_in_system_all_orders).tconfint_mean()))
 
    
    
    
    #sensitivity analysis
    sen_analysis[0]=['compeleted_orders',mean(compeleted_orders)]
    sen_analysis[1]=['utility_A',mean(utility_A)]
    sen_analysis[2]=['utility_B',mean(utility_B)]
    sen_analysis[3]=['utility_C',mean(utility_C)]
    sen_analysis[4]=['mean_lq_A',mean(mean_lq_A)]
    sen_analysis[5]=['mean_lq_B',mean(mean_lq_B)]
    sen_analysis[6]=['mean_lq_C',mean(mean_lq_C)]
    sen_analysis[7]=['mean_lq_Cord1',mean(mean_lq_Cord1)]
    sen_analysis[8]=['mean_lq_Cord2',mean(mean_lq_Cord2)]
    sen_analysis[9]=['mean_wq_A',mean(mean_wq_A)]
    sen_analysis[10]=['mean_wq_B',mean(mean_wq_B)]
    sen_analysis[11]=['mean_wq_C',mean(mean_wq_C)]
    sen_analysis[12]=['mean_wq_Cord1',mean(mean_wq_Cord1)]
    sen_analysis[13]=['mean_wq_Cord2',mean(mean_wq_Cord2)]
    sen_analysis[14]=['mean_being_in_system_order1',mean(mean_being_in_system_order1)]
    sen_analysis[15]=['mean_being_in_system_order2',mean(mean_being_in_system_order2)]
    sen_analysis[16]=['mean_being_in_system_all_orders',mean(mean_being_in_system_all_orders)]
     
        
        
#main_code
max_fel = 0         # Maximum length that FEL get (the current event does not count in)
header_list = ['Step', 'Current Event', 'Clock', 'Server A','Server B','Server C', 'Queue A','Queue B','Queue Cord1','Queue Cord2','Stock Order1','Stock Order2','q1','q2',
              'Compeleted orders','Served by A','Served by B','Served by Cord1','Served by Cord2','Server busy time A','Server busy time B',
              'Server busy time C','waiting in queue A','waiting in queue B','waiting in queue Cord1','waiting in queue Cord2',
              'time weighted length A','time weighted length B','time weighted length Cord1','time weighted length Cord2',
              'total being in system order1','total being in system order2', 'stock_num A', 'stock_num B']     # Headers at First   
                       
arr_B=1
work_A=1
rep_A=1
serv_C=1 #global variables for sensitivity analysis

simulation(int(input("enter the simulation time:")))

enter the simulation time:24


# Sensitivity analysis

 #sensitivity analysis on interarrival time for order2

In [16]:
#note that before sensitivity analysis we should comment all this section

arr_B=0.1#seting the target parameter to 0.1 and increasing  that in loop
work_A=1
rep_A=1
serv_C=1
sen_analysis=[[0,0]]*17 #defining 2D list
workbook = xw.Workbook("C:/Users/System-Pc/Desktop/handy/sut/ترم 7/simulation/پروژه/فاز 2/Sensitivity analysis_XW.xlsx")  
# Open a New Workbook (Simultaneously Created an Excel) #note that if you want run this in your computer you should
                                                                  #change the address
            
worksheet = workbook.add_worksheet(" arr_B analysis")   # Add a Sheet
   
for i in range(20):
    arr_B=(i*0.1)+0.1
    simulation(24)
    for j in range(17):
        worksheet.write(0,0,'step')
        worksheet.write(i+1,0,i+1)
        worksheet.write(0,j+1,sen_analysis[j][0])
        worksheet.write(i+1,j+1,sen_analysis[j][1]) #writing the values
    excel_formatting(workbook, worksheet,i+1)         # Format the Sensitivity analysis Excel sheet arr_B



#sensitivity analysis on working time of A

In [17]:
arr_B=1
work_A=0.1 #seting the target parameter to 0.1 and increasing  that in loop
rep_A=1
serv_C=1
sen_analysis=[[0,0]]*17 #defining 2D list
                 
worksheet = workbook.add_worksheet("work_A analysis")   # Add a Sheet
   
for i in range(20):
    work_A=(i*0.1)+0.1
    simulation(24)
    for j in range(17):
        worksheet.write(0,0,'step')
        worksheet.write(i+1,0,i+1)
        worksheet.write(0,j+1,sen_analysis[j][0])
        worksheet.write(i+1,j+1,sen_analysis[j][1]) #writing the values
        excel_formatting(workbook, worksheet,i+1)         # Format the Sensitivity analysis Excel sheet work_A


#sensitivity analysis on repairing time of A

In [18]:
arr_B=1
work_A=1
rep_A=0.1 #seting the target parameter to 0.1 and increasing  that in loop
serv_C=1
sen_analysis=[[0,0]]*17 #defining 2D list
               
worksheet = workbook.add_worksheet("rep_A analysis")   # Add a Sheet
   
for i in range(20):
    rep_A=(i*0.1)+0.1
    simulation(24)
    for j in range(17):
        worksheet.write(0,0,'step')
        worksheet.write(i+1,0,i+1)
        worksheet.write(0,j+1,sen_analysis[j][0])
        worksheet.write(i+1,j+1,sen_analysis[j][1])
        excel_formatting(workbook, worksheet,i+1)        # Format the Sensitivity analysis Excel sheet rep_A
 

#sensitivity analysis on service time of C

In [19]:
arr_B=1
work_A=1
rep_A=1
serv_C=0.1 #seting the target parameter to 0.1 and increasing  that in loop
sen_analysis=[[0,0]]*17 #defining 2D list
                  
worksheet = workbook.add_worksheet("serv_C analysis")   # Add a Sheet
   
for i in range(20):
    serv_C=(i*0.1)+0.1
    simulation(24)
    for j in range(17):
        worksheet.write(0,0,'step')
        worksheet.write(i+1,0,i+1)
        worksheet.write(0,j+1,sen_analysis[j][0])
        worksheet.write(i+1,j+1,sen_analysis[j][1])
        excel_formatting(workbook, worksheet,i+1)         # Format the Sensitivity analysis Excel sheet serv_C
workbook.close()              # Close the workbook  